In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.6.0


In [3]:
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]', 'You ever want something ', "that you know you shouldn't have ", "The more you know you shouldn't have it, ", 'The more you want it ', 'And then one day you get it, ', "It's so good too ", "But it's just like my girl "]


In [4]:
import re
import numpy as np
import tensorflow as tf

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Spoken Intro:]
You ever want something 
that you know you shouldn't have 
The more you know you shouldn't have it, 
The more you want it 
And then one day you get it, 
It's so good too 
But it's just like my girl 


In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

In [18]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    filtered_tensor = []
    for seq in tensor:
        if len(seq) <= 15:
            filtered_tensor.append(seq)
    
    filtered_tensor = tf.keras.preprocessing.sequence.pad_sequences(filtered_tensor, padding='post', maxlen=15)
    
    return filtered_tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

In [20]:
from sklearn.model_selection import train_test_split

# 훈련 데이터와 평가 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(tensor, tensor, test_size=0.2)

# 결과 출력
print("enc_train shape:", enc_train.shape)
print("enc_val shape:", enc_val.shape)
print("dec_train shape:", dec_train.shape)
print("dec_val shape:", dec_val.shape)

enc_train shape: (124810, 15)
enc_val shape: (31203, 15)
dec_train shape: (124810, 15)
dec_val shape: (31203, 15)


In [39]:
print(enc_train[:20, :])

[[    2   667  1140   183   140    12     3     0     0     0     0     0
      0     0     0]
 [    2   309    56    55    37    15   292    18     7     3     0     0
      0     0     0]
 [    2    36    31     6  1034    10   100     7    54   287     3     0
      0     0     0]
 [    2    23   145   751    10   645     3     0     0     0     0     0
      0     0     0]
 [    2  4497   248    20     3     0     0     0     0     0     0     0
      0     0     0]
 [    2     8 10984   193  1511  1306   397     8   133  1028     3     0
      0     0     0]
 [    2  4140  2718     1    20     3     0     0     0     0     0     0
      0     0     0]
 [    2   236     4     8     5    61    53   225     3     0     0     0
      0     0     0]
 [    2  1312   658     7    29     3     0     0     0     0     0     0
      0     0     0]
 [    2   446    47   215    16    14    13   776     3     0     0     0
      0     0     0]
 [    2    36    65    16    38    58    93    17 

In [40]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [44]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
train_src_input = enc_train[:, :-1]  
val_src_input = enc_val[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
train_tgt_input = dec_train[:, 1:]    
val_tgt_input = dec_val[:, 1:]    
print(train_src_input[0])
print(val_src_input[0])
print(train_tgt_input[0])
print(val_tgt_input[0])

[   2  667 1140  183  140   12    3    0    0    0    0    0    0    0]
[  2   5  22  18  13  85  10  64  17  80  19 216   3   0]
[ 667 1140  183  140   12    3    0    0    0    0    0    0    0    0]
[  5  22  18  13  85  10  64  17  80  19 216   3   0   0]


In [67]:
BUFFER_SIZE = len(train_src_input)
BATCH_SIZE = 256
steps_per_epoch = len(train_src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_src_input, train_tgt_input))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [68]:
BUFFER_SIZE = len(val_src_input)
BATCH_SIZE = 256
steps_per_epoch = len(val_src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((val_src_input, val_tgt_input))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [83]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 512 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 2048 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [137]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for train_src_sample, train_tgt_sample in train_dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(train_src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ -2.399432 ,   3.4696307, -11.795337 , ..., -10.625549 ,
         -10.421461 , -11.764851 ],
        [ -5.7951274,   4.0280704, -12.897945 , ...,  -8.704094 ,
          -8.085398 , -12.91992  ],
        [ -3.4825697,   7.796071 , -10.128011 , ..., -12.637671 ,
          -9.987819 , -10.391852 ],
        ...,
        [ 16.744806 ,   5.3439574, -17.437952 , ..., -14.947506 ,
         -14.934046 , -17.492596 ],
        [ 19.73973  ,   4.793841 , -18.466776 , ..., -15.542299 ,
         -14.51721  , -18.659498 ],
        [ 21.636787 ,   4.56243  , -18.77944  , ..., -15.771791 ,
         -13.586947 , -19.065031 ]],

       [[ -2.399432 ,   3.4696307, -11.795337 , ..., -10.625549 ,
         -10.421461 , -11.764851 ],
        [ -2.971292 ,   5.40478  ,  -7.52974  , ...,  -6.304164 ,
          -7.757793 ,  -7.4714046],
        [ -7.2699647,   5.128393 ,  -8.849103 , ...,  -8.882236 ,
         -10.719846 ,  -8.777653 ],
        .

In [72]:
model.summary()

Model: "text_generator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_14 (LSTM)               multiple                  6295552   
_________________________________________________________________
lstm_15 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_7 (Dense)              multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [85]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.

from keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# 모델 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=[early_stopping])

Epoch 1/30
487/487 [==============================] - 243s 495ms/step - loss: 3.2697 - val_loss: 2.8841
Epoch 2/30
487/487 [==============================] - 241s 495ms/step - loss: 2.7231 - val_loss: 2.6287
Epoch 3/30
487/487 [==============================] - 241s 495ms/step - loss: 2.4015 - val_loss: 2.4450
Epoch 4/30
487/487 [==============================] - 241s 495ms/step - loss: 2.0815 - val_loss: 2.3080
Epoch 5/30
487/487 [==============================] - 241s 495ms/step - loss: 1.7827 - val_loss: 2.2126
Epoch 6/30
487/487 [==============================] - 242s 496ms/step - loss: 1.5229 - val_loss: 2.1517
Epoch 7/30
487/487 [==============================] - 242s 496ms/step - loss: 1.3156 - val_loss: 2.1269
Epoch 8/30
487/487 [==============================] - 242s 496ms/step - loss: 1.1636 - val_loss: 2.1299
Epoch 9/30
487/487 [==============================] - 242s 496ms/step - loss: 1.0678 - val_loss: 2.1432
Epoch 10/30
487/487 [==============================] - 242s 496m

In [86]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [133]:
generate_text(model, tokenizer, init_sentence="<start> i love") # 시작문장으로 i love를 넣어 문장생성 함수 실행

'<start> i love you , i love you , i love you <end> '

In [134]:
generate_text(model, tokenizer, init_sentence="<start> kiss") # 시작문장으로 kiss를 넣어 문장생성 함수 실행

'<start> kiss girls like katy perry <end> '

Katy Perry 언급까지. I kissed a girl 이란 노래를 아는 거니?